# Init

In [1]:
import configparser, os, sys

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import kaiser

config = "bruz"
if config == "ur1":
    sys.path.insert(0, "../")
    siepelDir = "C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2"
elif config == "bruz":
    sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
    siepelDir = "/home/pleroy/DATA/CAMILL/WR15_v2"

import toolsdiadem.tools as dia
import toolsdiadem.ser as ser
from toolsdiadem.ConfigSer import ConfigSer
from toolsdiadem.Gate import Gate
from toolsdiadem.Probe import Probe

In [2]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2

In [3]:
cmap='jet'
configSupport = os.path.join(siepelDir, "wr15_plaqueSupport.ini")
configRef = os.path.join(siepelDir, "wr15_plaqueRef.ini")
conf = ConfigSer(configSupport)
confRef = ConfigSer(configRef)
conf.printConfig()
confRef.printConfig()

centerFreq = 62.5
bandWidth = 12.4
idxElevation = 20
valMin = -90
vmin = -90
vmax = -40

/home/pleroy/DATA/CAMILL/WR15_v2/wr15_plaqueSupport.ini
nb_elev 41
nb_freq 2001
nb_ssb 3
el0 0.7
az0 -0.44
base_path /home/pleroy/DATA/CAMILL/WR15_v2
out_path /home/pleroy/DATA/CAMILL/WR15_v2/processing/wr15
gateWidth 500.0
gateCenter 6450.0
/home/pleroy/DATA/CAMILL/WR15_v2/wr15_plaqueRef.ini
nb_elev 41
nb_freq 2001
nb_ssb 3
el0 0.31
az0 -0.44
base_path /home/pleroy/DATA/CAMILL/WR15_v2
out_path /home/pleroy/DATA/CAMILL/WR15_v2/processing/wr15
gateWidth 0.0
gateCenter 0.0


In [4]:
withPlots = 0

## Fond de chambre

In [5]:
file_f2c = f"{conf.out_path}/data/WR15_S11_fond_de_chambre.txt"
tmp = np.loadtxt(file_f2c, skiprows=1)
print(file_f2c)
S11_f2c = tmp[:,1] + 1j * tmp[:,2]

/home/pleroy/DATA/CAMILL/WR15_v2/processing/wr15/data/WR15_S11_fond_de_chambre.txt


## Probe

In [7]:
file_probe = os.path.join(conf.base_path, "S11FDC_WR15.s1p")
probe = Probe(file_probe)
print(file_probe)

/home/pleroy/DATA/CAMILL/WR15_v2/S11FDC_WR15.s1p


## Plaque de référence

In [8]:
file_ref = f"{confRef.out_path}/data/WR15_S11_plaque_ref.txt"
tmp = np.loadtxt(file_ref, skiprows=1)
S11_reference = tmp[:, 1] + 1j * tmp[:, 2]
freq_reference = tmp[:, 0]
print(file_ref)
d = np.linspace(0, 300 / (freq_reference[1] - freq_reference[0]), conf.nb_freq) * 0.5

/home/pleroy/DATA/CAMILL/WR15_v2/processing/wr15/data/WR15_S11_plaque_ref.txt


In [40]:
fig, ax = plt.subplots(1,1)
ax.plot(d, ser.dB(np.fft.ifft(S11_reference)), label="reference")
ax.plot(d, ser.dB(np.fft.ifft(S11_f2c)), label="f2c")
ax.plot(d, ser.dB(np.fft.ifft(probe.S11)), label="probe")
ax.plot(d, ser.dB(np.fft.ifft(S11_reference - probe.S11)), label="reference - probe")
ax.plot(d, ser.dB(np.fft.ifft(S11_reference - S11_f2c)), label="reference - f2c")
ax.grid()
ax.legend()
ax.set_xlabel("[m]")
ax.set_ylabel("[dB]")
ax.set_title(f"time domain, width {conf.gateWidth:.0f} center {conf.gateCenter:.0f}")

Text(0.5, 1.0, 'time domain, width 500 center 6450')

# Réflectivité
ref: ser4siepel_multiband_scan_elev_Reflectivite.sce

## Initialisation des variables

In [41]:
data_S11         = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
data_S11_td      = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
data_S11_tg      = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
S11_minus_moy    = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
S11_minus_moy_td = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
S11_minus_moy_tg = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
reflectivite     = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
S11_moy          = np.zeros(conf.nb_freq,            dtype=complex)
S11_moy_square   = np.zeros(conf.nb_freq,            dtype=complex)

## Lecture des données et time gating

In [42]:
myGate = Gate(d, width=conf.gateWidth, center=conf.gateCenter)
gate = myGate.gateLLC

gateSize 167


## Calcul de la réflectivité

In [54]:
date, absorbant = "20200701", "000_A"
#date, absorbant = "20200703", "013_0"
dir_abs = f"{conf.base_path}/{date}/{absorbant}"    
freq_all, data_S11 = ser.getData_ssb(conf.nb_elev, conf.nb_freq, conf.nb_ssb, data_dir)

d = np.linspace(0, 300 / (freq_all[1] - freq_all[0]), conf.nb_freq) * 0.5

In [55]:
reflectivite = ser.getReflectivite(data_S11, S11_reference, S11_f2c, myGate.gateLLC)

In [56]:
S11_plaque_ref_tg = np.fft.fft(np.fft.ifft(S11_reference - S11_f2c) * gate)
S11_minus_fdc = data_S11 - S11_f2c
s11_minus_fdc_td = np.fft.ifft(S11_minus_fdc, axis=1)
s11_minus_fdc_tg = s11_minus_fdc_td * gate
S11_minus_fdc_tg = np.fft.fft(s11_minus_fdc_tg, axis=1)
reflectivite = S11_minus_fdc_tg / S11_plaque_ref_tg

s11_td = np.fft.ifft(data_S11, axis=1)
s11_td_plaqueRef = np.fft.ifft(S11_reference)

In [57]:
idx = 20
plt.figure()
plt.plot(d, ser.dB(s11_td[idx, :]), label="s11_td")
plt.plot(d, ser.dB(s11_td_plaqueRef), label="s11_td_plaqueRef")
plt.plot(d, ser.dB(s11_minus_fdc_td[idx, :]), label="s11_minus_fdc_td")
plt.plot(d, ser.dB(s11_minus_fdc_tg[idx, :]), label="s11_minus_fdc_tg")
plt.legend()
plt.grid()
plt.title(f"{absorbant} {date}, élévation {conf.elevation[idx]}°")

Text(0.5, 1.0, '000_A 20200701, élévation 0.7°')

## Plot

In [33]:
toPlot = np.zeros( (conf.nb_elev, conf.nb_freq) )
gatebis  = 0.0 * myGate.gateLLC
gatebis[np.where(np.abs(freq_all - centerFreq) < bandWidth)] = 1.0
toPlot = 20 * np.log10(1e-12 + np.abs(reflectivite * gatebis))

valMin = -90
toPlot = 0.5 * (np.abs(toPlot - valMin) + toPlot - valMin) + valMin
valMax = np.amax(toPlot)
valMin = np.amin(toPlot)
print( f"valMax = {valMax:.2f} valMin = {valMin:.2f}")
     
vmin = -80
vmax = -0
fig, ax = plt.subplots(1, 1)
im = plt.pcolormesh(conf.elevation, np.flip(freq_all), np.rot90(toPlot), cmap=cmap)

title = f"Réflectivité absorbant\n{data_dir}"
ax.set_title(title)
ax.set_xlabel("Elévation [°]")
ax.set_ylabel("Fréquence [GHz]")
dia.addColorBar( im, ax, aspect='auto')

valMax = -36.59 valMin = -90.00


# Tous les absorbants

20200701 = ["000_A", "001_C", "002_E", "003_W", "004_V", "005_G"]

20200703 = ["006_L", "007_M", "008_X", "009_H", "010_I", "011_K", "012_J", "013_O", "014_N", "015_D", "016_T", "017_AB_u", "018_N_u"]

In [34]:
np.save(f"{conf.out_path}/data/freq_WR15", freq_all)
np.save(f"{conf.out_path}/data/elevation_WR15", conf.elevation)

In [46]:
idx = np.where(np.abs(freq_all - centerFreq) < bandWidth)
idxAlt = np.where(np.abs(freq_all - centerFreq) >= bandWidth)

dic_stats = {}
toPlot = np.zeros((conf.nb_elev, conf.nb_freq))

gate = myGate.gateLLC
date = "20200703" # 01, 03

for absorbant in conf.dic_dir[date]:

    seq = 0
    flag_gate = 1
    data_dir = f"{conf.base_path}/{date}/{absorbant}"
    descriptor = f"{absorbant}_{date}"
    
    print(f"======= {absorbant} =======")

    for ielev in range(conf.nb_elev):
        freq_all, S11_all = ser.getFreqAndS11(data_dir, conf.nb_ssb, seq0=seq, verbose=0)
        seq = seq + 3
        data_S11[ ielev, : ] = S11_all
    
    S11_plaque_ref_tg = np.fft.fft(np.fft.ifft(S11_reference - S11_f2c) * gate)
    tab_freq = freq_all
    S11_minus_moy    = data_S11 - S11_f2c
    S11_minus_moy_td = np.fft.ifft(S11_minus_moy, axis=1)
    S11_minus_moy_tg = np.fft.fft(S11_minus_moy_td * gate, axis=1)

    #=============
    # REFLECTIVITE
    reflectivite = np.zeros((conf.nb_elev, conf.nb_freq), dtype=complex)
    reflectivite = S11_minus_moy_tg / S11_plaque_ref_tg
    np.save( 
        f"{conf.out_path}/data/reflectivite/{descriptor}",
        reflectivite 
    )
    
    # drop lower and higher frequencies values
    toPlot[:,idx] = 20 * np.log10(1e-12 + np.abs(reflectivite[:,idx]))
    toPlot[:,idxAlt] = -90

    fig, ax = plt.subplots( 1, 1 )
    im = ax.pcolormesh(conf.elevation, np.flip(freq_all), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax)
    title = f"Réflectivité [{absorbant}]"
    ax.set_title(title)
    ax.set_xlabel("Elévation [°]")
    ax.set_ylabel("Fréquence [GHz]")
    dia.addColorBar(im, ax, aspect='auto')
    fig.savefig(
        f"{conf.out_path}/plot/reflectivite/reflectivite_{descriptor}.png"
    )

    #======
    # STATS
    valMax = np.amax(toPlot)
    valMin = np.amin(toPlot)
    print(f"valMax = {valMax:.2f} valMin = {valMin:.2f}")

    absReflectivite   = np.abs(reflectivite)
    reflectiviteMoy   = np.average(absReflectivite[:,idx])
    reflectiviteSigma = np.std(absReflectivite[:,idx])

    reflectiviteMoy_dB   = 20 * np.log10(np.abs(reflectiviteMoy))
    reflectiviteSigma_dB = 20 * np.log10(np.abs(reflectiviteSigma))
    print(f"moy = {reflectiviteMoy:.6f}, sigma = {reflectiviteSigma:.6f}")
    print(f"moy = {reflectiviteMoy_dB:.2f} [dB], sigma = {reflectiviteSigma_dB:.2f} [dB]")
    
    #==========
    # HISTOGRAM
    fig, ax = plt.subplots(1,1)
    ax.hist((absReflectivite[:,idx]).flatten(), 50, label="histogramme (valeurs linéaires)")
    ax.grid()
    title = f"Réflectivité, histogramme [{absorbant}]"
    ax.set_title(title)
    ax.legend()
    fig.savefig(
        f"{conf.out_path}/plot/hist/hist_{descriptor}.png"
    )
    
    #=====
    # PLOT
    fig, ax = plt.subplots(1,1)
    ax.plot( freq_all, toPlot[ idxElevation, : ] )
    ax.grid()
    title = f"Réflectivité [{absorbant}]"
    ax.set_title(title)
    ax.set_xlabel("f [GHz]")
    ax.set_ylabel("réflectivité [dB]")
    fig.savefig(
        f"{conf.out_path}/plot/reflectivite_vs_f/reflectivite_vs_f_{descriptor}.png"
    )
    
    plt.close("all")
    
    dic_stats[f"{descriptor}"] = (valMax, valMin, reflectiviteMoy, reflectiviteSigma, reflectiviteMoy_dB)

======= 006_L =======
valMax = -54.12 valMin = -130.19
moy = 0.000458, sigma = 0.000224
moy = -66.79 [dB], sigma = -73.00 [dB]
======= 007_M =======
valMax = -58.86 valMin = -118.24
moy = 0.000326, sigma = 0.000161
moy = -69.73 [dB], sigma = -75.87 [dB]
======= 008_X =======
valMax = -58.54 valMin = -109.10
moy = 0.000403, sigma = 0.000233
moy = -67.90 [dB], sigma = -72.67 [dB]
======= 009_H =======
valMax = -57.81 valMin = -116.49
moy = 0.000449, sigma = 0.000215
moy = -66.95 [dB], sigma = -73.34 [dB]
======= 010_I =======
valMax = -55.47 valMin = -120.00
moy = 0.000541, sigma = 0.000309
moy = -65.34 [dB], sigma = -70.20 [dB]
======= 011_K =======
valMax = -61.61 valMin = -123.72
moy = 0.000234, sigma = 0.000122
moy = -72.61 [dB], sigma = -78.25 [dB]
======= 012_J =======
valMax = -56.76 valMin = -122.96
moy = 0.000437, sigma = 0.000255
moy = -67.19 [dB], sigma = -71.87 [dB]
======= 013_O =======
valMax = -56.60 valMin = -111.67
moy = 0.000501, sigma = 0.000257
moy = -66.01 [dB], sigm

In [47]:
for absorbant in dic_stats.keys():
    stats = dic_stats[absorbant]
    print(f"{absorbant} {stats[0]:.2f} {stats[1]:.2f} {stats[2]:.6f} {stats[3]:.6f} {stats[4]:.2f}")

006_L_20200703 -54.12 -130.19 0.000458 0.000224 -66.79
007_M_20200703 -58.86 -118.24 0.000326 0.000161 -69.73
008_X_20200703 -58.54 -109.10 0.000403 0.000233 -67.90
009_H_20200703 -57.81 -116.49 0.000449 0.000215 -66.95
010_I_20200703 -55.47 -120.00 0.000541 0.000309 -65.34
011_K_20200703 -61.61 -123.72 0.000234 0.000122 -72.61
012_J_20200703 -56.76 -122.96 0.000437 0.000255 -67.19
013_O_20200703 -56.60 -111.67 0.000501 0.000257 -66.01
014_N_20200703 -55.65 -111.53 0.000709 0.000303 -62.99
015_D_20200703 -65.15 -136.85 0.000163 0.000086 -75.76
016_T_20200703 -61.47 -121.44 0.000255 0.000121 -71.86
017_AB_u_20200703 -63.63 -124.15 0.000171 0.000097 -75.36
018_N_u_20200703 -63.95 -119.73 0.000177 0.000094 -75.06
